In [ ]:
"""
look for an isekai anime from 2022
look for an isekai anime with female harem
look for an anime like attack on titan
look for an anime like bleach and fullmetal alchemist
look for an sports anime with ecchi



request -> json
fix genre and tags
look for genre and tags

get genre and tags from like-animes
full search
if 0? 
"""

In [23]:
from anilist import search_anime

search_results = search_anime(
    search_term="ghost in the shell",
    # genres=["Sports", "Ecchi"],  # Using single genre parameter
    # tags=["Volleyball"],
    per_page=5
)

# # Print all results first
print("All results:")
for anime in search_results:
    print(f"- {anime['title']['english'] or anime['title']['romaji']}")
    print(f"  Genres: {', '.join(anime['genres'])}")
    print(f"  Tags: {', '.join([tag['name'] for tag in anime['tags']])}")
    print("-" * 60)

All results:
- Ghost in the Shell
  Genres: Action, Psychological, Sci-Fi
  Tags: Cyberpunk, Artificial Intelligence, Philosophy, Urban, Robots, Cyborg, Police, Primarily Male Cast, Female Protagonist, Primarily Adult Cast, Seinen, Noir, Dystopian, Crime, Guns, Espionage, Nudity, Gore, CGI
------------------------------------------------------------
- Ghost in the Shell: Stand Alone Complex
  Genres: Action, Mystery, Psychological, Sci-Fi
  Tags: Cyberpunk, Police, Politics, Episodic, Robots, Philosophy, Crime, Artificial Intelligence, Cyborg, Work, Urban, Military, Detective, Seinen, Conspiracy, Primarily Male Cast, Female Protagonist, Guns, Primarily Adult Cast, Dystopian, Memory Manipulation, Real Robot, Ensemble Cast, CGI, Body Swapping, Kuudere, Bisexual, Gore
------------------------------------------------------------
- Ghost in the Shell: Stand Alone Complex 2nd GIG
  Genres: Action, Mystery, Psychological, Sci-Fi
  Tags: Police, Cyberpunk, Politics, Terrorism, Military, Philos

In [1]:
from typing import Optional, List, Literal
from pydantic import BaseModel
from crewai import Agent, Task, LLM
import os

# Initialize LLM (using GPT-4o from OpenAI)
llm = LLM(
    model="gpt-4.1",
    api_key=os.getenv('OPENAI_API_KEY'),
    max_tokens=2000,
    temperature=0
)

# 1 ── strict schema
class AnimeSearchParams(BaseModel):
    search_term: Optional[str] = None
    season:     Optional[Literal["WINTER", "SPRING", "SUMMER", "FALL"]] = None
    year:       Optional[int]  = None
    genres:     Optional[List[str]] = None
    tags:       Optional[List[str]] = None
    sort:       Optional[str] = None
    page:       Optional[int] = None
    per_page:   Optional[int] = None

    model_config = {"extra": "forbid"}

# 2 ── agent
mapper = Agent(
    name="AniListRequestMapper",
    role="Filter extractor",
    goal="Return only the filters in the user's request, as minimal JSON.",
    backstory="An anime librarian who knows the difference between genres and tags.",
    allow_delegation=False,
    llm=llm
)

# 3 ── task  – strict rules for genre vs tag + no nulls
OFFICIAL_GENRES = [
  "Action",
  "Adventure",
  "Comedy",
  "Drama",
  "Ecchi",
  "Fantasy",
  "Hentai",
  "Horror",
  "Mahou Shoujo",
  "Mecha",
  "Music",
  "Mystery",
  "Psychological",
  "Romance",
  "Sci-Fi",
  "Slice of Life",
  "Sports",
  "Supernatural",
  "Thriller"
]

map_request = Task(
    description=(
        "USER_REQUEST:\n"
        "{user_request}\n\n"
        "Produce **one JSON object** that validates against AnimeSearchParams.\n"
        "Rules:\n"
        f"• Only items in this list may appear in `genres`: {OFFICIAL_GENRES}.\n"
        "• Any other descriptive phrase (moods, sub‑genres like 'School Life', "
        "adjectives like 'Wholesome') must go into `tags`.\n"
        "• Title‑case every word (e.g. 'school‑life' → 'School Life').\n"
        "Don't necessarily include genres unless they are specified in the request.\n"
        "• Omit every key whose value would be null.\n\n"
        "Example:\n"
        "USER_REQUEST:  Recommend a dark fantasy from 2020.\n"
        "OUTPUT: {\"genres\": [\"Fantasy\"], \"tags\": [\"Dark\"], \"year\": 2020}"
    ),
    expected_output="A JSON dict with only the mentioned filters, no nulls.",
    output_json=AnimeSearchParams,
    agent=mapper,
)

In [5]:
from crewai import Crew, Process
from anilist import search_anime

# 4 ── quick demo -----------------------------------------------------------
prompt = "I want an isekai anime with a female harem"
prompt = "I want an sportsz anime and also ecchi"


crew = Crew(
    agents=[mapper],
    tasks=[map_request],
    process=Process.sequential,
)
crew.kickoff(inputs={"user_request": prompt})

params = {k: v for k, v in map_request.output.json_dict.items() if v is not None}
print(params)

from inspect import signature
assert set(params).issubset(signature(search_anime).parameters)

{'genres': ['Sports', 'Ecchi']}


In [16]:
"""
AniList Search Tool for CrewAI
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
This module exposes a CrewAI‐compatible tool that lets an agent search
AniList for anime by title, season, year, genres, tags, and sort order.
"""

from typing import List, Optional, Dict, Any, Type
import requests
from pydantic import BaseModel, Field
from crewai.tools import BaseTool


# --------------------------------------------------------------------------- #
#  Low‑level helper – unchanged from your original implementation
# --------------------------------------------------------------------------- #
def _fetch_from_anilist(query: str, variables: Dict[str, Any]) -> Dict[str, Any]:
    """Send a GraphQL request to the AniList public API."""
    url = "https://graphql.anilist.co"
    resp = requests.post(url, json={"query": query, "variables": variables})
    if resp.status_code != 200:
        raise RuntimeError(
            f"AniList query failed (HTTP {resp.status_code}): {resp.text}"
        )
    return resp.json()


# --------------------------------------------------------------------------- #
#  Pydantic input schema – every argument is documented for the LLM
# --------------------------------------------------------------------------- #
class SearchAnimeToolInput(BaseModel):
    """Schema for the AniList search tool."""
    search_term: Optional[str] = Field(
        None, description="Keyword to search in titles (Romaji or English)."
    )
    season: Optional[str] = Field(
        None, description='Season filter: "WINTER", "SPRING", "SUMMER", or "FALL".'
    )
    year: Optional[int] = Field(
        None, description="Year that the season aired (e.g. 2024)."
    )
    genre: Optional[str] = Field(
        None, description="A single genre to filter by."
    )
    genres: Optional[List[str]] = Field(
        None, description="A list of genres to filter by."
    )
    tags: Optional[List[str]] = Field(
        None, description="A list of AniList tags to filter by."
    )
    sort: Optional[List[str]] = Field(
        ["POPULARITY_DESC"],
        description='List of AniList sort keys, e.g. ["SCORE_DESC"].',
    )
    page: int = Field(
        1, ge=1, description="Page number of the result set (starts at 1)."
    )
    per_page: int = Field(
        20, ge=1, le=50, description="Results per page (max 50)."
    )


# --------------------------------------------------------------------------- #
#  CrewAI tool definition
# --------------------------------------------------------------------------- #
class SearchAnimeTool(BaseTool):
    """
    CrewAI tool: search_anime
    -------------------------
    A high‑level search against the AniList catalogue.  Returns a list of
    anime objects with id, titles, genres, tags, averageScore, episodes,
    format, status, seasonYear, and coverImage.
    """

    name: str = "search_anime"
    description: str = (
        "Search AniList for anime using free‑text title keywords, season/year, "
        "genres, tags, and sort order. Returns an array of matching anime."
    )
    args_schema: Type[BaseModel] = SearchAnimeToolInput

    # ‑‑‑‑ core execution ‑‑‑‑ #
    def _run(  # noqa: N802 (CrewAI naming convention)
        self,
        search_term: Optional[str] = None,
        season: Optional[str] = None,
        year: Optional[int] = None,
        genre: Optional[str] = None,
        genres: Optional[List[str]] = None,
        tags: Optional[List[str]] = None,
        sort: Optional[List[str]] = None,
        page: int = 1,
        per_page: int = 20,
    ) -> List[Dict[str, Any]]:
        """
        Perform the AniList query and return the raw media list.

        CrewAI automatically validates inputs with the Pydantic schema,
        so this method receives typed arguments.
        """
        query = """
        query (
          $search: String, $season: MediaSeason, $seasonYear: Int,
          $genre: String, $genres: [String], $tags: [String],
          $page: Int, $perPage: Int, $sort: [MediaSort]
        ) {
          Page(page: $page, perPage: $perPage) {
            media(
              search: $search, type: ANIME, season: $season,
              seasonYear: $seasonYear, genre: $genre, genre_in: $genres,
              tag_in: $tags, sort: $sort
            ) {
              id
              title { romaji english }
              genres
              tags { id name rank isMediaSpoiler }
              averageScore
              episodes
              format
              status
              seasonYear
              coverImage { medium }
            }
          }
        }
        """

        # Build variables map (include only non‑None keys)
        variables: Dict[str, Any] = {
            "page": page,
            "perPage": per_page,
        }
        if search_term:
            variables["search"] = search_term
        if season:
            variables["season"] = season
        if year:
            variables["seasonYear"] = year
        if genre:
            variables["genre"] = genre
        if genres:
            variables["genres"] = genres
        if tags:
            variables["tags"] = tags
        if sort:
            variables["sort"] = sort

        # Fire the request
        data = _fetch_from_anilist(query, variables)
        print("--------------BEGIN OF CALL------------------")
        print("query")
        print(query)
        print("--------------------------------")
        print("variables")
        print(variables)
        print("--------------------------------")
        print("data")
        print(data)
        print("--------------------------------")
        print(data["data"]["Page"]["media"])
        print("""data["data"]["Page"]["media"]""")
        print("--------------------------------")
        print("-END OF CALL-")
        return data["data"]["Page"]["media"]

    # Optional: define async version if your crew uses asynchronous execution
    async def _arun(self, **kwargs) -> Any:  # noqa: D401
        """Asynchronous wrapper around _run."""
        return self._run(**kwargs)

In [17]:
from crewai import Agent, Crew, Task

anime_tool = SearchAnimeTool()

anime_researcher = Agent(
    role="Anime Researcher",
    goal="Find animes that match query obtained from the user request",
    backstory="You are a seasoned anime critic.",
    tools=[anime_tool],
)

# (Optional but typical) — wrap the work you want done in a Task
recommendation_task = Task(
    description=(
        "Using AniList, compile a list of five animes that match the query."
    ),
    expected_output="Recommendations for animes that match the query. Eeach one with a small justification.",
    agent=anime_researcher,
)

# crew = Crew(tasks=[recommendation_task], agents=[anime_researcher])

# # Kick off the crew, passing any placeholders the task needs
# result = crew.kickoff(inputs={"query": params})
# print(result)


In [22]:
crew = Crew(
    agents=[mapper, anime_researcher],
    tasks=[map_request, recommendation_task],
    process=Process.sequential,
)

result = crew.kickoff(inputs={"user_request": "I want an anime like ghost in the shell"})
print(result)

--------------BEGIN OF CALL------------------
query

        query (
          $search: String, $season: MediaSeason, $seasonYear: Int,
          $genre: String, $genres: [String], $tags: [String],
          $page: Int, $perPage: Int, $sort: [MediaSort]
        ) {
          Page(page: $page, perPage: $perPage) {
            media(
              search: $search, type: ANIME, season: $season,
              seasonYear: $seasonYear, genre: $genre, genre_in: $genres,
              tag_in: $tags, sort: $sort
            ) {
              id
              title { romaji english }
              genres
              tags { id name rank isMediaSpoiler }
              averageScore
              episodes
              format
              status
              seasonYear
              coverImage { medium }
            }
          }
        }
        
--------------------------------
variables
{'page': 1, 'perPage': 5, 'genres': ['Action', 'Mystery', 'Psychological', 'Sci-Fi', 'Thriller'], 'tags': 